In [356]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True

01:43PM 39s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 163 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



2024-11-07 13:43:39.951 python[3309:1681169] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-07 13:43:39.951 python[3309:1681169] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [7]:
default_options = Dict(
        cut_l='1000um',
        cut_h='1200um',
        gap='55um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '30um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '200um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

## Calculate Qubit Params

In [8]:
import Transmon_property as tp

In [10]:
cap = 80*dp.u.fF
fq, (w1,alpha) = tp.freq_from_cap(cap)

In [13]:
fq.to(dp.u.GHz)

<Quantity 4.69305741 GHz>

In [14]:
alpha.to(dp.u.MHz)

<Quantity 242.12786656 MHz>

In [395]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

# Define the transmon parameters
C = 60 * u.fF  # Capacitance in femtofarads
L = 13 * u.nH  # Inductance in nanohenrys
# Calculate the flux quantum Phi0
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

# Calculate the Josephson energy EJ and charging energy EC
EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
EC = (c.e.si**2 / (2 * C)/c.h).to(u.GHz)
ratio = 70
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    #, equivalencies=u.spectral())  # Convert to GHz
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Print calculated EJ and EC
    # print(f"Calculated EJ (GHz): {EJ}")
    # print(f"Calculated EC (GHz): {EC}")

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]

# Print the results
    # print(f"Transition frequencies (GHz): {transition_frequencies}")
    # print(f"Anharmonicity (GHz): {anharmonicity}")
    # print('Ej/Ec', EJ/EC)
    return transition_frequencies[0]#EJ,anharmonicity

# Plot the energy levels
# plt.figure(figsize=(8, 6))
# plt.plot(eigenenergies, 'bo-', label='Eigenenergies')
# plt.xlabel('Energy Level Index')
# plt.ylabel('Energy (GHz)')
# plt.title('Transmon Eigenenergies')
# plt.legend()
# plt.grid(True)
# plt.show()

In [133]:
calculate_qubit_freq(10*u.nH)

5.281322018983346

In [134]:
target_function(10,5)

0.28132201898334586

In [174]:
from scipy.optimize import root_scalar

# Define the calculate_qubit_frequency function

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency



In [175]:
def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

### Function to calculate capacitance, inductance based on qubit frequency

In [181]:
# Desired qubit frequency
def qubit_freq_to_cap(fq, ratio = 65):
    desired_frequency = fq   # Desired frequency in GHz

    # Use scipy.optimize.root_scalar to find the root
    result = root_scalar(target_function, args=(desired_frequency,ratio), bracket=[1, 100], method='brentq')

    # Print the result
    if result.converged:
        L_optimal = result.root
        print(f"Optimal inductance L: {L_optimal} nH")
    else:
        print("Root finding did not converge")
    c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
    return c_optimal, L_optimal*u.nH

In [182]:
qubit_freq_to_cap(3.5*u.GHz, ratio = 65)

Optimal inductance L: 15.630232544091612 nH


(<Quantity 120.39211514 fF>, <Quantity 15.63023254 nH>)

In [264]:
def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency

class TransmonQubit():
    def __init__(self, freq, ratio):
        self.freq = freq
        self.ratio = ratio
        self.C = None,
        self.L = None
        self.EJ = None
        self.EC = None
        self.update_params()
        self.alpha = self.EC
        
    def update_params(self):
        self.C, self.L = qubit_freq_to_cap(self.freq, ratio = self.ratio)
        Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

        # Calculate the Josephson energy EJ and charging energy EC
        EJ = (((Phi0/2/np.pi)**2/self.L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
        EC = (c.e.si**2 / (2 * self.C)/c.h).to(u.GHz)
        self.EJ = EJ
        self.EC = EC
        
    def get_params(self):
        return [self.freq, self.ratio, self.qubit_capacitance, self.qubit_inductance, self.EJ, self.EC]
    def get_qcap(self):
        return self.C
    
    def get_qind(self):
        return self.L
    
    def get_EJ(self):
        return self.EJ
    
    def get_EC(self):
        return self.EC
    
    def qubit_freq_to_cap(self, ):
        desired_frequency = self.fq   # Desired frequency in GHz
        ratio = self.ratio
        # Use scipy.optimize.root_scalar to find the root
        result = root_scalar(target_function, args=(desired_frequency,ratio), bracket=[1, 100], method='brentq')

        # Print the result
        if result.converged:
            L_optimal = result.root
            print(f"Optimal inductance L: {L_optimal} nH")
        else:
            print("Root finding did not converge")
        c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
        return c_optimal, L_optimal*u.nH

In [265]:
qubit = TransmonQubit(6*u.GHz, ratio = 65)

Optimal inductance L: 9.117635650720107 nH


In [139]:
result_L = []
result_cap = []
for desired_frequency in np.linspace(3,6,100):
    result = root_scalar(target_function, args=(desired_frequency,), bracket=[1, 100], method='brentq')

    # Print the result
    if result.converged:
        L_optimal = result.root
        result_L += [L_optimal]
        result_cap += [inductance_to_cap(L_optimal*u.nH).to(u.fF).value]
        # print(f"Optimal inductance L: {L_optimal} nH")
    else:
        print("Root finding did not converge")

In [140]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
x = np.linspace(3,6,100)
y1 = result_L
y2 = result_cap

# Create the figure and the first y-axis
fig, ax1 = plt.subplots()

# Plot the first variable
ax1.plot(x, y1, 'b-', label='sin(x)')
ax1.set_xlabel('Target Frequency (GHz)')
ax1.set_ylabel('Inductance Value (nH)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create the second y-axis and plot the second variable
ax2 = ax1.twinx()
ax2.plot(x, y2, 'r--', label='cos(x)')
ax2.set_ylabel('Capacitance (fF)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Add grid and title
ax1.grid(True)
plt.title('Plot with Two Different Y-Axes')

# Show the plot
plt.show()

In [141]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [142]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [131]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
x = np.linspace(3,6,100)
y1 = np.array(result_L)
y2 = np.array(result_cap)

j_size = unitless_find_j_area(y1)
c_size = unitless_find_pad_len(y2)

# Create the figure and the first y-axis
fig, ax1 = plt.subplots()

# Plot the first variable
ax1.plot(x, j_size, 'b-', label='sin(x)')
ax1.set_xlabel('Target Frequency (GHz)')
ax1.set_ylabel('Junction Size (um2)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create the second y-axis and plot the second variable
ax2 = ax1.twinx()
ax2.plot(x, c_size, 'r--', label='cos(x)')
ax2.set_ylabel('Pad length (um)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Add grid and title
ax1.grid(True)
plt.title('Ej/Ec = 65')
plt.savefig('Ej_Ec_65.png')
# Show the plot
plt.show()

In [114]:
find_junction_area(10*u.nH, Jc = 0.42*u.uA/u.um**2)

<Quantity 0.07835857 um2>

In [118]:
pad_len_from_C(80*u.fF)

19.362899999999996

In [143]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    

In [146]:
get_cap_params(np.array([[80,0,0,0],[0,80,0,0],[0,0,80,0],[0,0,0,80]]),0.5)

[<Quantity 0.48425573 GHz>, <Quantity 0. MHz>, 40.0, inf]

In [147]:
def g_from_cap(cap):
    return 8*convert_capacitance_to_energy(cap).to(u.MHz)

In [153]:
(g_from_cap(50)**2/2/u.GHz).to(u.MHz)

<Quantity 4802.63403635 MHz>

In [157]:
(1/np.sqrt(500*u.us/300/u.MHz /np.pi/2).si).to(u.MHz)

<Quantity 1.94162591 MHz>

## Resonator calculations

In [207]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance

In [208]:
import qiskit_metal.analyses as analyses

In [226]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

In [229]:
def f(line_gap):
    return(analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)[4]-50)
optimized_gap = optimize.fsolve(f,line_gap.si.value)[0]*u.m
optimized_gap.to(u.um)

In [ ]:
freq = 6*u.GHz.si
line_width = line_width
line_gap = optimized_gap
substrate_thickness = sub_t
film_thickness = metal_t
dielectric_constant = epsilon
loss_tangent = 0.01
london_penetration = 50*u.nm


Lk, Lext,Cap,G_,Z0,etf,Cstar = analyses.cpw_calculations.lumped_cpw(freq = freq.to(u.Hz).value,
                                     line_width=line_width.to(u.m).value,
                                     line_gap=line_gap.to(u.m).value,
                                     substrate_thickness=substrate_thickness.to(u.m).value,
                                     film_thickness=film_thickness.to(u.m).value,
                                     london_penetration_depth=london_penetration.to(u.m).value,)
Lk*=u.H
Lext*=u.H
Cap*=u.F
G_*=u.S
Z0*=u.Ohm
etf*=u.H/u.m
Cstar*=u.F



In [238]:
target_length = analyses.cpw_calculations.guided_wavelength(freq = freq.to(u.Hz).value,
                                     line_width=line_width.to(u.m).value,
                                     line_gap=line_gap.to(u.m).value,
                                     substrate_thickness=substrate_thickness.to(u.m).value,
                                     film_thickness=film_thickness.to(u.m).value,)[0]*u.m

In [246]:
def param_from_res_freq(freq, short_on_one_end = False):
    Lk, Lext,Cap,G_,Z0,etf,Cstar = analyses.cpw_calculations.lumped_cpw(freq = freq.to(u.Hz).value,
                                     line_width=line_width.to(u.m).value,
                                     line_gap=line_gap.to(u.m).value,
                                     substrate_thickness=substrate_thickness.to(u.m).value,
                                     film_thickness=film_thickness.to(u.m).value,
                                     london_penetration_depth=london_penetration.to(u.m).value,)
    Lk*=u.H
    Lext*=u.H
    Cap*=u.F
    G_*=u.S
    Z0*=u.Ohm
    etf*=u.H/u.m
    Cstar*=u.F
    target_length = analyses.cpw_calculations.guided_wavelength(freq = freq.to(u.Hz).value,
                                     line_width=line_width.to(u.m).value,
                                     line_gap=line_gap.to(u.m).value,
                                     substrate_thickness=substrate_thickness.to(u.m).value,
                                     film_thickness=film_thickness.to(u.m).value,)[0]*u.m
    if short_on_one_end:
        target_length /= 4
    else:
        target_length /= 2
    return Lk+Lext,Cap,target_length

In [498]:
class ReadoutResonator():
    def __init__(self, freq, short_on_one_end = False,optimal_pin = None, optimal_gap = None ):
        self.freq = freq
        self.short_on_one_end = short_on_one_end
        self.substrate_thickness = 350*u.um
        self.film_thickness = 100*u.nm
        if (optimal_gap is None) and (optimal_pin is None):
            self.pin = 15*u.um
            self.optimize_gap()
        elif optimal_gap is None:
            self.pin = optimal_pin
            self.optimize_gap()
        elif optimal_pin is None:
            self.gap = optimal_gap
            self.optimize_pin()
        
        self.param_from_res_freq(freq, short_on_one_end = short_on_one_end)
        
    def get_params(self):
        return [self.L, self.C, self.target_length]
    
    def get_L(self):
        return self.L
    
    def get_C(self):
        return self.C
    
    def get_target_length(self):
        return self.target_length
    
    def param_from_res_freq(self,freq, short_on_one_end = False):
        self.Lk, self.Lext,self.C,self.G,self.Z0,self.etf,self.Cstar = analyses.cpw_calculations.lumped_cpw(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)
        self.Lk*=u.H
        self.Lext*=u.H
        self.C*=u.F
        self.G*=u.S
        self.Z0*=u.Ohm
        self.etf*=u.H/u.m
        self.Cstar*=u.F
        target_length = analyses.cpw_calculations.guided_wavelength(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)[0]*u.m
        if short_on_one_end:
            target_length /= 4
        else:
            target_length /= 2
        self.len = target_length
    def optimize_gap(self):
        def f(line_gap):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            self.pin.to(u.m).value,
                                            line_gap,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_gap = optimize.fsolve(f,self.pin.si.value)[0]*u.m
        self.gap = optimized_gap.to(u.um)
    def optimize_pin(self):
        def f(line_width):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            line_width,
                                            self.gap.si.value,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_pin = optimize.fsolve(f,self.gap.si.value)[0]*u.m
        self.pin = optimized_pin.to(u.um)

In [499]:
res = ReadoutResonator(6*u.GHz, short_on_one_end = False, optimal_pin = 20*u.um)

In [500]:
res.optimize_pin()

In [502]:
res.gap

<Quantity 11.10942503 um>

In [286]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)

Optimal inductance L: 13.676453476080171 nH


In [312]:
(c.e.si**2*qubit.freq/c.hbar/res.C).si 

<Quantity 5.81899891e+15 1 / s2>

In [288]:
alpha = qubit.EC
delta = qubit.freq-res.freq
T1_limit = 5*u.ms

In [323]:
def chi(C, qubit, resonator):
    g_val = g(C,qubit, resonator)
    delta = (qubit.freq-resonator.freq)*2*np.pi
    sum_freq = (qubit.freq+resonator.freq)*2*np.pi
    alpha = qubit.EC
    return 2*g_val**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
# def kappa(alpha,g,delta):
#     '''optimal SNR requires kappa approx 2 chi'''
#     return 2*chi(alpha,g,delta)

In [324]:
g(50*u.fF,qubit,res).to(u.MHz)

<Quantity 187.66334404 MHz>

In [327]:
chi(30*u.fF,qubit,res).to(u.MHz)

<Quantity 0.03114174 MHz>

Args:
        freq (float): The frequency of interest, in Hz
        C_in (float): Effective capacitance between CPW and environment (from Q3D), in Farads
        freq_res (float): Lowest resonant frequency of a CPW (from HFSS), in Hz
        length (float): Length of the CPW readout resonator, in meters
        res_width (float): Width of the resonator trace (center) line, in meters
        res_gap (float): Width of resonator gap (dielectric space), in meters
        eta (float): 2.0 for half-wavelength resonator; 4.0 for quarter-wavelength resonator

    Returns:
        float: Kappa value
    """

In [345]:
c = 30*u.fF

In [408]:
def kappa(c_tr,res,):
    return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value,
                                       c_tr.si.value,
                                       res.freq.si.value)*u.Hz).to(u.MHz)

In [353]:
kappa(30*u.fF,res)

<Quantity 0.30939721 MHz>

In [371]:
c_qr = 40*u.fF
c_tr = 25*u.fF

purcellT1(c_qr,c_tr,qubit,res).to(u.ms)

<Quantity 5.18975772 ms>

In [373]:
kappa_val = kappa(c_tr,res)
chi_val = chi(c_qr,qubit,res)
print(kappa_val.to(u.MHz))
print(chi_val.to(u.MHz))

(kappa_val/chi_val).si

0.21485917317405873 MHz
0.055363096881049116 MHz


<Quantity 3.88090958>

In [374]:
qubit.alpha

<Quantity 0.18387753 GHz>

<Quantity 4.97985918 ms>

In [404]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)



Optimal inductance L: 13.676453476080171 nH


In [405]:
c_qr = optimal_cqr(5*u.ms,qubit,res)
purcellT1q(c_qr,qubit,res).to(u.ms)

<Quantity 5.02183879 ms>

In [406]:
chi(c_qr,qubit,res).to(u.MHz)

<Quantity 0.07839968 MHz>

In [419]:
kappa_val = 2*chi(c_qr,qubit,res).to(u.MHz)

In [420]:
kappa_val

<Quantity 0.15679936 MHz>

In [415]:
optimal_ctr(2*chi(c_qr,qubit,res).si,qubit,res)

<Quantity 21.4 fF>

In [416]:
kappa(25*u.fF,res)

<Quantity 0.21485917 MHz>

In [457]:
class DispersiveReadout():
    def __init__(self, qubit, res, T1, c_qr = None, c_tr = None):
        self.qubit = qubit
        self.res = res
        self.pT1 = T1
        self.optimize_cqr()
        self.optimize_ctr()
        
    def get_params(self):
        return [self.qubit, self.res, self.c_qr, self.c_tr]
    
    def get_cqr(self):
        return self.c_qr
    
    def get_ctr(self):
        return self.c_tr
    
    def get_qubit(self):
        return self.qubit
    
    def get_res(self):
        return self.res
    def calc_g(self):
        c_qr = self.c_qr
        Cq = self.qubit.C
        Cr = self.res.C
        wq = self.qubit.freq*np.pi*2
        wr = self.res.freq*np.pi*2
        EJ = self.qubit.EJ
        EC = self.qubit.EC
        return c_qr/Cq*np.sqrt(c.e.si**2*wr/c.hbar/Cr)*(EJ/8/EC)**0.25
    def calc_kappa(self):
        res = self.res
        c_tr = self.c_tr
        return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value,
                                       c_tr.si.value,
                                       res.freq.si.value)*u.Hz).to(u.MHz)
    def calc_chi(self):
        qubit = self.qubit
        resonator = self.res
        g_val = self.calc_g()
        delta = (qubit.freq-resonator.freq)*2*np.pi
        sum_freq = (qubit.freq+resonator.freq)*2*np.pi
        alpha = qubit.EC
        return 2*g_val**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
    def calc_purcellT1(self,):
        qubit = self.qubit
        res = self.res
        kappa_val = self.calc_kappa()
        g_val = self.calc_g()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        purcellGamma = g_val**2*kappa_val/delta**2
        
        return 1/purcellGamma/2/np.pi
    def calc_purcellT1q(self,):
        '''taking kappa = 2*chi for the optimal SNR, can directly get the T1q'''
        c_qr = self.c_qr
        qubit = self.qubit
        res = self.res
        g_val = self.calc_g()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        chi_val = self.calc_chi()
        kappa_val = chi_val*2
        
        purcellGamma = g_val**2*kappa_val/delta**2
        
        return 1/purcellGamma/2/np.pi
    def optimize_cqr(self,):
        '''find the optimal coupling capacitance'''
        target_t1 = self.pT1
        c_qr = 1*u.fF
        self.c_qr = c_qr
        while self.calc_purcellT1q().to(u.ms) > 1.01*target_t1:
            c_qr += .1*u.fF
            self.c_qr = c_qr
        return c_qr
    def optimize_ctr(self, target_kappa=None):
        '''find the optimal coupling capacitance'''
        if target_kappa is None:
            target_kappa = self.calc_chi()*2
        c_tr = 1*u.fF
        self.c_tr = c_tr
        while kappa(c_tr,res) < target_kappa:
            c_tr += .1*u.fF
            self.c_tr = c_tr
        # return c_tr


In [461]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [466]:
dr.calc_g().to(u.MHz)

<Quantity 178.65550352 MHz>

In [468]:
dr.calc_chi().to(u.MHz)

<Quantity 0.07839968 MHz>

In [470]:
dr.c_tr

<Quantity 21.4 fF>

In [474]:
qubit.EC

<Quantity 0.18387753 GHz>

In [170]:
alpha = 150*u.MHz
delta = 2*u.GHz
T1 = 5*u.ms

chi = np.sqrt(alpha/2/T1*np.pi*2).to(u.MHz)

g = np.sqrt(chi*alpha/delta*(delta+alpha)/2).to(u.MHz)

print(chi)
print(g)

0.3069980123839465 MHz
4.97510952125234 MHz


In [173]:
w_r = 6.5*u.GHz
Zr = 50 * u.ohm
kappa = chi*2

C = (kappa*2/Zr/w_r**2).to(u.fF)
print(C)


0.5812980116145732 fF


In [478]:
import numpy as np
import matplotlib.pyplot as plt

def mandelbrot(width, height, x_min=-2.0, x_max=1.0, y_min=-1.5, y_max=1.5, max_iter=100):
    """
    Generate the Mandelbrot set and return it as a 2D array.

    Parameters:
        width (int): Width of the output array (number of pixels in the x direction).
        height (int): Height of the output array (number of pixels in the y direction).
        x_min (float): Minimum x-coordinate of the view.
        x_max (float): Maximum x-coordinate of the view.
        y_min (float): Minimum y-coordinate of the view.
        y_max (float): Maximum y-coordinate of the view.
        max_iter (int): Maximum number of iterations to determine divergence.

    Returns:
        np.ndarray: 2D array representing the Mandelbrot set.
    """
    # Create an array to store the Mandelbrot set data
    mandelbrot_set = np.zeros((height, width))

    # Create a grid of complex numbers over the specified range
    x_values = np.linspace(x_min, x_max, width)
    y_values = np.linspace(y_min, y_max, height)

    for i, y in enumerate(y_values):
        for j, x in enumerate(x_values):
            c = complex(x, y)
            z = 0
            iteration = 0

            # Iterate to determine if the point is in the Mandelbrot set
            while abs(z) <= 2 and iteration < max_iter:
                z = z*z + c
                iteration += 1

            # Store the iteration count (color intensity)
            mandelbrot_set[i, j] = iteration

    return mandelbrot_set

# Plotting function for the Mandelbrot set
def plot_mandelbrot(mandelbrot_set):
    plt.imshow(mandelbrot_set, cmap='twilight', extent=(-2, 1, -1.5, 1.5))
    plt.colorbar(label="Iteration Count")
    plt.title("Mandelbrot Set")
    plt.xlabel("Re(c)")
    plt.ylabel("Im(c)")
    plt.show()

# Generate and plot the Mandelbrot set
width, height = 1000, 1000
mandelbrot_set = mandelbrot(width, height)
plot_mandelbrot(mandelbrot_set)
